### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [93]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [94]:
#calculate values using built in pandas functions
school_count=school_data_complete["school_name"].nunique()
student_count=school_data_complete["Student ID"].nunique()
total_budget=school_data["budget"].sum()
average_math_score=student_data["math_score"].mean()
average_reading_score=student_data["reading_score"].mean()

#calculating passing average percentage by querying the students with scores above 70, counting them, 
#and dividing by the total count. Multiplication by 100 is done for displaying %
passing_math_percent=student_data.loc[student_data["math_score"]>=70,"Student ID"].nunique()/student_count*100
passing_reading_percent=student_data.loc[student_data["reading_score"]>=70,"Student ID"].nunique()/student_count*100

#average of the previous two, as stated in the homework instructions
#altough, in my opinion the overall passing rate should not be calculated like this
overall_passing_rate=(passing_math_percent+passing_reading_percent)/2

# I think that the corect way of calculating overall_passing_rate is the following:
# overall_passing_rate=student_data.loc[
#      student_data["math_score"] & student_data["reading_score"]>=70,
#      "Student ID"].nunique()/student_count

#creating a dataframe for displaying summary values
district_summary=pd.DataFrame({
    "Total Schools":[school_count],
    "Total Students":[student_count],
    "Total Budget":[total_budget],
    "Average Math Score":[average_math_score],
    "Average Reading Score":[average_reading_score],
    "% Passing Math":[passing_math_percent],
    "% Passing Reading":[passing_reading_percent],
    "% Overall Passing Rate":[overall_passing_rate]
})

#mapping values to display correct units
district_summary["Total Students"]=district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"]=district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["% Passing Math"]=district_summary["% Passing Math"].map("%{:.6f}".format)
district_summary["% Passing Reading"]=district_summary["% Passing Reading"].map("%{:.6f}".format)
district_summary["% Overall Passing Rate"]=district_summary["% Overall Passing Rate"].map("%{:.6f}".format)

district_summary
district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,%74.980853,%85.805463,%80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [95]:
#defining base table
school_summary=school_data[["school_name","type","budget"]]

#counting the total students by school, and adding to the base table
group_by_school=school_data_complete.groupby(["school_name"]).count()
school_summary=pd.merge(school_summary,group_by_school["Student ID"],on="school_name")

#renaming columns
school_summary=school_summary.rename(columns={
    "school_name":"School Name","type":"School Type","budget":"Total School Budget","Student ID":"Total Students"})

#adding column with budget per student to the base table and formatting
school_summary["Per Student Budget"]=school_summary["Total School Budget"]/school_summary["Total Students"]

#adding average math & reading score per school to the base table and renaming columns
group_by_school=school_data_complete.groupby(["school_name"]).mean()
school_summary=pd.merge(school_summary,group_by_school[["math_score","reading_score"]],left_on="School Name",right_on="school_name")
school_summary=school_summary.rename(columns={"math_score":"Average Math Score","reading_score":"Average Reading Score"})

#adding column with the percent of students passing math (score>70) to the base table
dquerry_math_pass=school_data_complete["math_score"]>=70
count_math_pass_byschool=school_data_complete[dquerry_math_pass].groupby("school_name")["math_score"].count()
school_summary=pd.merge(school_summary,count_math_pass_byschool,left_on="School Name",right_on="school_name")
school_summary["math_score"]=school_summary["math_score"]/school_summary["Total Students"]*100
school_summary=school_summary.rename(columns={"math_score":"% Passing Math"})

#adding column with the percent of students passing reading (score>70) to the base table
dquerry_reading_pass=school_data_complete["reading_score"]>=70
count_math_pass_byschool=school_data_complete[dquerry_reading_pass].groupby("school_name")["reading_score"].count()
school_summary=pd.merge(school_summary,count_math_pass_byschool,left_on="School Name",right_on="school_name")
school_summary["reading_score"]=school_summary["reading_score"]/school_summary["Total Students"]*100
school_summary=school_summary.rename(columns={"reading_score":"% Passing Reading"})

#adding column with the overall passing rate to the base table
school_summary["Overall Passing Rate"]=(school_summary["% Passing Reading"]+school_summary["% Passing Math"])/2

#replacing column index with "School Name"
school_summary=school_summary.set_index("School Name")

#formatting columns
school_summary["Total School Budget"]=school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"]=school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary["% Passing Math"]=school_summary["% Passing Math"].map("%{:,.2f}".format)
school_summary["% Passing Reading"]=school_summary["% Passing Reading"].map("%{:,.2f}".format)
school_summary["Overall Passing Rate"]=school_summary["Overall Passing Rate"].map("%{:,.2f}".format)

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [96]:
school_summary=school_summary.sort_values(["Overall Passing Rate"], ascending=False)
school_summary.head()

,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,83.061895,83.975780,%94.13,%97.04,%95.59
Thomas High School,Charter,"$1,043,130.00",1635,$638.00,83.418349,83.848930,%93.27,%97.31,%95.29
Griffin High School,Charter,"$917,500.00",1468,$625.00,83.351499,83.816757,%93.39,%97.14,%95.27
Pena High School,Charter,"$585,858.00",962,$609.00,83.839917,84.044699,%94.59,%95.95,%95.27
Wilson High School,Charter,"$1,319,574.00",2283,$578.00,83.274201,83.989488,%93.87,%96.54,%95.20


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [97]:
school_summary=school_summary.sort_values(["Overall Passing Rate"], ascending=True)
school_summary.head()

,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"$2,547,363.00",3999,$637.00,76.842711,80.744686,%66.37,%80.22,%73.29
Figueroa High School,District,"$1,884,411.00",2949,$639.00,76.711767,81.158020,%65.99,%80.74,%73.36
Huang High School,District,"$1,910,635.00",2917,$655.00,76.629414,81.182722,%65.68,%81.32,%73.50
Johnson High School,District,"$3,094,650.00",4761,$650.00,77.072464,80.966394,%66.06,%81.22,%73.64
Ford High School,District,"$1,763,916.00",2739,$644.00,77.102592,80.746258,%68.31,%79.30,%73.80


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [98]:
#group all data by school and grade
math_gr_school_grade=school_data_complete.groupby(["school_name","grade"])["math_score"].mean()

#create dataframe by joining first 2 columns and rename columns
math_scores_by_grade=pd.merge(math_gr_school_grade.loc[:,"9th"],math_gr_school_grade.loc[:,"10th"],on="school_name")
math_scores_by_grade=math_scores_by_grade.rename(columns={"math_score_x":"9th","math_score_y":"10th"})

#add the rest of the columns
math_scores_by_grade['11th']=math_gr_school_grade.loc[:,"11th"]
math_scores_by_grade['12th']=math_gr_school_grade.loc[:,"12th"]

#format columns
math_scores_by_grade["9th"]=math_scores_by_grade["9th"].map("{:,.2f}".format)
math_scores_by_grade["10th"]=math_scores_by_grade["10th"].map("{:,.2f}".format)
math_scores_by_grade["11th"]=math_scores_by_grade["11th"].map("{:,.2f}".format)
math_scores_by_grade["12th"]=math_scores_by_grade["12th"].map("{:,.2f}".format)
math_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [99]:
#group all data by school and grade
reading_gr_school_grade=school_data_complete.groupby(["school_name","grade"])["reading_score"].mean()

#create dataframe by joining first 2 columns and rename columns
reading_scores_by_grade=pd.merge(reading_gr_school_grade.loc[:,"9th"],reading_gr_school_grade.loc[:,"10th"],on="school_name")
reading_scores_by_grade=reading_scores_by_grade.rename(columns={"reading_score_x":"9th","reading_score_y":"10th"})

#add the rest of the columns
reading_scores_by_grade['11th']=reading_gr_school_grade.loc[:,"11th"]
reading_scores_by_grade['12th']=reading_gr_school_grade.loc[:,"12th"]

#format columns
reading_scores_by_grade["9th"]=reading_scores_by_grade["9th"].map("{:,.2f}".format)
reading_scores_by_grade["10th"]=reading_scores_by_grade["10th"].map("{:,.2f}".format)
reading_scores_by_grade["11th"]=reading_scores_by_grade["11th"].map("{:,.2f}".format)
reading_scores_by_grade["12th"]=reading_scores_by_grade["12th"].map("{:,.2f}".format)
reading_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [100]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [101]:
#creating bins by school name
school_spending_bins=pd.cut(pd.to_numeric(school_summary["Per Student Budget"].str.strip("$,")),spending_bins,labels=group_names)
#school_spending_summary=pd.merge(school_data_complete,school_spending_bins,left_on="school_name",right_on="School Name")

#adding bins to the base table and renaming column added
school_spending_summary=pd.merge(school_summary,school_spending_bins,left_on="School Name",right_on="School Name")
school_spending_summary=school_spending_summary.rename(columns={"Per Student Budget_y":"Spending Ranges (Per Student)"})

#removing formatting and converting to number for fields of interest
school_spending_summary["% Passing Math"]=pd.to_numeric(school_spending_summary["% Passing Math"].str.strip("%,"))
school_spending_summary["% Passing Reading"]=pd.to_numeric(school_spending_summary["% Passing Reading"].str.strip("%,"))
school_spending_summary["Overall Passing Rate"]=pd.to_numeric(school_spending_summary["Overall Passing Rate"].str.strip("%,"))

#grouping data requested by the bins
group_by_spending_bins=school_spending_summary.groupby(["Spending Ranges (Per Student)"])["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"].mean()
group_by_spending_bins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460000,96.610000,95.035000
$585-615,83.599686,83.885211,94.230000,95.900000,95.065000
$615-645,79.079225,81.891436,75.668333,86.106667,80.886667
$645-675,76.997210,81.027843,66.163333,81.133333,73.650000


## Scores by School Size

* Perform the same operations as above, based on school size.

In [102]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [103]:
#creating bins by school name
school_size_bins=pd.cut(school_summary["Total Students"],size_bins,labels=group_names)
#school_size_summary=pd.merge(school_data_complete,school_size_bins,left_on="school_name",right_on="School Name")

#adding bins to the base table and renaming column added
school_size_summary=pd.merge(school_summary,school_size_bins,left_on="School Name",right_on="School Name")
school_size_summary=school_size_summary.rename(columns={"Total Students_y":"School Size"})

#removing formatting and converting to number for fields of interest
school_size_summary["% Passing Math"]=pd.to_numeric(school_size_summary["% Passing Math"].str.strip("%,"))
school_size_summary["% Passing Reading"]=pd.to_numeric(school_size_summary["% Passing Reading"].str.strip("%,"))
school_size_summary["Overall Passing Rate"]=pd.to_numeric(school_size_summary["Overall Passing Rate"].str.strip("%,"))

#grouping data requested by the bins
group_by_spending_bins=school_size_summary.groupby(["School Size"])["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"].mean()
group_by_spending_bins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.55000,96.10000,94.82500
Medium (1000-2000),83.374684,83.864438,93.59800,96.79000,95.19600
Large (2000-5000),77.746417,81.344493,69.96375,82.76625,76.36375


## Scores by School Type

* Perform the same operations as above, based on school type.

In [104]:
school_type_summary=school_summary

#removing formatting and converting to number for fields of interest
school_type_summary["% Passing Math"]=pd.to_numeric(school_summary["% Passing Math"].str.strip("%,"))
school_type_summary["% Passing Reading"]=pd.to_numeric(school_summary["% Passing Reading"].str.strip("%,"))
school_type_summary["Overall Passing Rate"]=pd.to_numeric(school_summary["Overall Passing Rate"].str.strip("%,"))

#grouping data requested by the bins
school_type_summary=school_type_summary.groupby(["School Type"])["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"].mean()
school_type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620000,96.586250,95.103750
District,76.956733,80.966636,66.548571,80.798571,73.672857
